In [93]:
from physionet2023.modeling.scoringUtil import compute_challenge_score, roc_auc_score, regression_to_probability
import torch
from physionet2023.dataProcessing.patientDatasets import MetadataOnlyDataset
import os
import glob
import pandas as pd
metadata_ds = MetadataOnlyDataset("../data")

In [94]:
pre_df = list()

for path in glob.glob("../cache/loo/ICARE_*"):
    pid = path.split("/")[-1].split(".")[0]
    _, patient_metadata, _ = metadata_ds.get_by_pid(pid)

    pre_df.append({
        "pid": pid,
        "label": int(int(patient_metadata["CPC"]) > 2),
        'label_raw': int(patient_metadata['CPC']),
        'pred_raw': torch.load(path) * 4 + 1,
        "preds": regression_to_probability(torch.load(path) * 4 + 1),
        "Age": float(patient_metadata["Age"]),
        "Sex": patient_metadata["Sex"],
        "ROSC": float(patient_metadata["ROSC"]),
        "TTM": float(patient_metadata["TTM"]),
        "OHCA": bool(patient_metadata["OHCA"] == 'True'),
        "VFib": bool(patient_metadata["VFib"] == 'True')
    })

df = pd.DataFrame(data=pre_df)
df = df.set_index("pid")
df

,label,label_raw,pred_raw,preds,Age,Sex,ROSC,TTM,OHCA,VFib
pid,,,,,,,,,,
ICARE_0423,1,5,"[[tensor(5.0960)], [tensor(5.1194)], [tensor(4...","[[tensor(1.)], [tensor(1.)], [tensor(0.9955)],...",71.0,Female,25.0,33.0,False,False
ICARE_0501,0,2,"[[tensor(2.7963)], [tensor(1.5004)], [tensor(2...","[[tensor(0.5593)], [tensor(0.1668)], [tensor(0...",54.0,Male,NaN,36.0,True,True
ICARE_0474,1,5,"[[tensor(2.3823)], [tensor(3.3218)], [tensor(2...","[[tensor(0.4608)], [tensor(0.6644)], [tensor(0...",64.0,Male,NaN,33.0,True,True
ICARE_0630,1,3,"[[tensor(3.1582)], [tensor(4.2176)], [tensor(4...","[[tensor(0.6316)], [tensor(0.8435)], [tensor(0...",50.0,Female,35.0,33.0,False,False
ICARE_0484,0,2,"[[tensor(4.2662)], [tensor(3.6415)], [tensor(3...","[[tensor(0.8532)], [tensor(0.7283)], [tensor(0...",81.0,Male,NaN,36.0,False,False
...,...,...,...,...,...,...,...,...,...,...
ICARE_0389,1,5,[[tensor(4.6845)]],[[tensor(0.9369)]],56.0,Male,NaN,NaN,False,False
ICARE_0351,0,1,"[[tensor(3.1648)], [tensor(3.3776)], [tensor(2...","[[tensor(0.6330)], [tensor(0.6755)], [tensor(0...",59.0,Male,18.0,33.0,True,True
ICARE_0612,1,5,"[[tensor(2.9962)], [tensor(3.4330)], [tensor(3...","[[tensor(0.5992)], [tensor(0.6866)], [tensor(0...",54.0,Male,NaN,33.0,True,True


In [95]:
df["p_mean"] = df['preds'].apply(lambda x: float(x.mean()))
df['p_median'] = df['preds'].apply(lambda x: float(x.median()))
df['p_max'] = df['preds'].apply(lambda x: float(x.max()))
df['p_min'] = df['preds'].apply(lambda x: float(x.min()))
df['p_25'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.25)))
df['p_75'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.75)))
df['p_40'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.40)))
df['p_60'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.60)))
df['p_std'] = df['preds'].apply(lambda x: float(x.std()))
df["pr_mean"] = df['pred_raw'].apply(lambda x: float(x.mean()))
df['pr_diff'] = (df['label_raw'] - df['p_median']).abs()


df

,label,label_raw,pred_raw,preds,Age,Sex,ROSC,TTM,OHCA,VFib,...,p_median,p_max,p_min,p_25,p_75,p_40,p_60,p_std,pr_mean,pr_diff
pid,,,,,,,,,,,,,,,,,,,,,
ICARE_0423,1,5,"[[tensor(5.0960)], [tensor(5.1194)], [tensor(4...","[[tensor(1.)], [tensor(1.)], [tensor(0.9955)],...",71.0,Female,25.0,33.0,False,False,...,1.000000,1.000000,0.776540,0.977166,1.000000,0.999097,1.000000,0.060078,4.985054,4.000000
ICARE_0501,0,2,"[[tensor(2.7963)], [tensor(1.5004)], [tensor(2...","[[tensor(0.5593)], [tensor(0.1668)], [tensor(0...",54.0,Male,NaN,36.0,True,True,...,0.521214,0.711742,0.082579,0.395053,0.613229,0.475936,0.560408,0.153513,2.580843,1.478786
ICARE_0474,1,5,"[[tensor(2.3823)], [tensor(3.3218)], [tensor(2...","[[tensor(0.4608)], [tensor(0.6644)], [tensor(0...",64.0,Male,NaN,33.0,True,True,...,0.402928,0.664356,0.092211,0.373645,0.512700,0.395814,0.466719,0.136811,2.339272,4.597072
ICARE_0630,1,3,"[[tensor(3.1582)], [tensor(4.2176)], [tensor(4...","[[tensor(0.6316)], [tensor(0.8435)], [tensor(0...",50.0,Female,35.0,33.0,False,False,...,0.743660,0.867474,0.409393,0.660750,0.792374,0.688973,0.755728,0.105340,3.592302,2.256340
ICARE_0484,0,2,"[[tensor(4.2662)], [tensor(3.6415)], [tensor(3...","[[tensor(0.8532)], [tensor(0.7283)], [tensor(0...",81.0,Male,NaN,36.0,False,False,...,0.649794,0.940364,0.136122,0.537315,0.719459,0.612257,0.669811,0.158013,3.192035,1.350206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ICARE_0389,1,5,[[tensor(4.6845)]],[[tensor(0.9369)]],56.0,Male,NaN,NaN,False,False,...,0.936908,0.936908,0.936908,0.936908,0.936908,0.936908,0.936908,NaN,4.684542,4.063092
ICARE_0351,0,1,"[[tensor(3.1648)], [tensor(3.3776)], [tensor(2...","[[tensor(0.6330)], [tensor(0.6755)], [tensor(0...",59.0,Male,18.0,33.0,True,True,...,0.608426,0.721424,0.514553,0.583610,0.660863,0.601316,0.630384,0.051185,3.090576,0.391574
ICARE_0612,1,5,"[[tensor(2.9962)], [tensor(3.4330)], [tensor(3...","[[tensor(0.5992)], [tensor(0.6866)], [tensor(0...",54.0,Male,NaN,33.0,True,True,...,0.718376,0.904238,0.432780,0.633574,0.773293,0.674638,0.743108,0.113709,3.518017,4.281624


In [106]:
def drop_outliers(preds):
    p_std = preds.std()
    p_med = preds.median()

    adjusted = torch.where(preds > (p_med + p_std), preds, p_med)
    return float(adjusted.mean())

df['p_drop_outliers'] = df['preds'].apply(drop_outliers)

df['p_drop_outliers']

pid
ICARE_0423    1.000000
ICARE_0501    0.534789
ICARE_0474    0.438092
ICARE_0630    0.752626
ICARE_0484    0.674033
                ...   
ICARE_0389    0.936908
ICARE_0351    0.629310
ICARE_0612    0.737047
ICARE_0504    0.435813
ICARE_0495    0.196845
Name: p_drop_outliers, Length: 288, dtype: float64

In [112]:
label = df['label'].to_numpy()
preds = df['p_drop_outliers'].to_numpy()

print(roc_auc_score(label, preds))
print(compute_challenge_score(label, preds))

0.7987288135593219
0.5176470588235295


In [66]:
def describe(df):
    for float_col in ["Age", "ROSC", "TTM", "p_std"]:
        print(f"{float_col} mean: {df[float_col].mean()}")
    
    for cat_col in ["Sex", "OHCA", "VFib"]:
        print(f"{cat_col} value counts:")
        print(df[cat_col].value_counts())


In [67]:
describe(df.nlargest(n=25, columns=['pr_diff']))

Age mean: 58.125
ROSC mean: 21.61111111111111
TTM mean: 33.3
p_std mean: 0.096815534401685
Sex value counts:
Male      16
Female     8
nan        1
Name: Sex, dtype: int64
OHCA value counts:
True     14
False    11
Name: OHCA, dtype: int64
VFib value counts:
True     13
False    12
Name: VFib, dtype: int64


In [68]:
describe(df.nsmallest(n=25, columns=['pr_diff']))

Age mean: 63.48
ROSC mean: 34.583333333333336
TTM mean: 33.142857142857146
p_std mean: 0.08604128059485684
Sex value counts:
Male      17
Female     8
Name: Sex, dtype: int64
OHCA value counts:
True     21
False     4
Name: OHCA, dtype: int64
VFib value counts:
False    14
True     11
Name: VFib, dtype: int64
